# Slicing

In this notebook I will divide each scan in slices along the X axis. These slices will server as units of our data for modelling.

File configuration

In [1]:
# Adjust the frac parameter for loess smoothing
# to control the degree of smoothing applied to the scan data.
diff_dir = "../scandata/difference loess frac=0.20"

Libraries used:

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import os

Let's load the pointclouds.

In [3]:
npy_files = [filename for filename in os.listdir(diff_dir) if filename.endswith(".npy")]
difference_point_clouds = [np.load(os.path.join(diff_dir, npy_file)) for npy_file in npy_files]

num_of_clouds = len(difference_point_clouds)
total_rows = 0

print(f"Loaded {num_of_clouds} difference point clouds.")

for i in range(num_of_clouds):
    print(f"Difference point cloud {npy_files[i]} shape: {difference_point_clouds[i].shape}")
    total_rows += difference_point_clouds[i].shape[0]
print(f"Total number of rows: {total_rows}")

Loaded 5 difference point clouds.
Difference point cloud 221111_144114__binary_onlylines114378to117271 loess frac=0.2_difference.npy shape: (5543532, 3)
Difference point cloud 221111_144114__binary_onlylines145900to148050 loess frac=0.2_difference.npy shape: (3786525, 3)
Difference point cloud 221111_144114__binary_onlylines25660to28100 loess frac=0.2_difference.npy shape: (3535680, 3)
Difference point cloud 221111_144114__binary_onlylines47178to50517 loess frac=0.2_difference.npy shape: (3414927, 3)
Difference point cloud 221111_144114__binary_onlylines86841to90142 loess frac=0.2_difference.npy shape: (4718054, 3)
Total number of rows: 20998718


In [4]:
# Function to slice point clouds by x-coordinate
def slice_pointcloud_by_x(point_cloud, slice_width=10):
    # Ensure shape is (N, 3)
    if point_cloud.shape[0] == 3 and point_cloud.shape[1] != 3:
        point_cloud = point_cloud.T
    x_coords = point_cloud[:, 0].astype(int)
    min_x = x_coords.min()
    max_x = x_coords.max()
    slices = []
    starts = []
    for start_x in range(min_x, max_x + 1, slice_width):
        end_x = start_x + slice_width
        mask = (x_coords >= start_x) & (x_coords < end_x)
        slice_points = point_cloud[mask]
        if slice_points.shape[0] > 0:
            slices.append(slice_points)
            starts.append(start_x)
    return slices, starts

Creating slices each 10 units wide.

In [5]:
all_slices = []
for idx, pc in enumerate(difference_point_clouds):
    slices, starts = slice_pointcloud_by_x(pc, slice_width=10)
    print(f"Point cloud {idx}: {len(slices)} slices")
    for i, start_x in enumerate(starts):
        print(f"  Slice {i}: starts at X={start_x}, points={slices[i].shape[0]}")
    all_slices.append(slices)

Point cloud 0: 290 slices
  Slice 0: starts at X=0, points=19875
  Slice 1: starts at X=10, points=20483
  Slice 2: starts at X=20, points=20329
  Slice 3: starts at X=30, points=20297
  Slice 4: starts at X=40, points=20249
  Slice 5: starts at X=50, points=20265
  Slice 6: starts at X=60, points=20289
  Slice 7: starts at X=70, points=20466
  Slice 8: starts at X=80, points=20389
  Slice 9: starts at X=90, points=20311
  Slice 10: starts at X=100, points=20354
  Slice 11: starts at X=110, points=20359
  Slice 12: starts at X=120, points=20266
  Slice 13: starts at X=130, points=20364
  Slice 14: starts at X=140, points=20436
  Slice 15: starts at X=150, points=20320
  Slice 16: starts at X=160, points=20386
  Slice 17: starts at X=170, points=20445
  Slice 18: starts at X=180, points=20346
  Slice 19: starts at X=190, points=20353
  Slice 20: starts at X=200, points=20367
  Slice 21: starts at X=210, points=20329
  Slice 22: starts at X=220, points=20360
  Slice 23: starts at X=230, 

The slices are of uneven size. I will pad them with 0's to make them the same size.

In [6]:
# Find the maximum number of points in any slice across all clouds
max_points = 0
for cloud_slices in all_slices:
    for s in cloud_slices:
        if s.shape[0] > max_points:
            max_points = s.shape[0]

# Pad each slice with zeros to have the same number of points (along axis 0)
padded_slices = []
for cloud_idx, cloud_slices in enumerate(all_slices):
    padded_cloud = []
    for slice_idx, s in enumerate(cloud_slices):
        n_points = s.shape[0]
        if n_points < max_points:
            # Pad with zeros
            pad_width = ((0, max_points - n_points), (0, 0))
            s_padded = np.pad(s, pad_width, mode='constant', constant_values=0)
        else:
            s_padded = s
        padded_cloud.append(s_padded)
    padded_slices.append(padded_cloud)

print(f"All slices are now padded to {max_points} points each.")


All slices are now padded to 20483 points each.


# Saving the slices

In [7]:
# Create output directory for slices
slices_dir = diff_dir.replace("difference", "slices")
os.makedirs(slices_dir, exist_ok=True)

# Save all slices of each point cloud into a single .npy file
for cloud_idx, (cloud_slices, fname) in enumerate(zip(padded_slices, npy_files)):
    base_name = os.path.splitext(fname)[0]
    # Stack slices into a 3D array: (num_slices, max_points, 3)
    stacked = np.stack(cloud_slices, axis=0)
    outname = f"{base_name}_slices.npy"
    outpath = os.path.join(slices_dir, outname)
    np.save(outpath, stacked)
    print(f"Saved: {outpath}")

Saved: ../scandata/slices loess frac=0.20/221111_144114__binary_onlylines114378to117271 loess frac=0.2_difference_slices.npy
Saved: ../scandata/slices loess frac=0.20/221111_144114__binary_onlylines145900to148050 loess frac=0.2_difference_slices.npy
Saved: ../scandata/slices loess frac=0.20/221111_144114__binary_onlylines25660to28100 loess frac=0.2_difference_slices.npy
Saved: ../scandata/slices loess frac=0.20/221111_144114__binary_onlylines47178to50517 loess frac=0.2_difference_slices.npy
Saved: ../scandata/slices loess frac=0.20/221111_144114__binary_onlylines86841to90142 loess frac=0.2_difference_slices.npy
